# Exercise: Twitter Network Analysis using Random Walk and CrossWalk

To identify important nodes and communities in a Twitter user dataset, we can combine the Random Walk and CrossWalk methods. Random Walk simulates random steps through a network to determine which nodes are visited more often, highlighting influential nodes. CrossWalk adjusts edge weights to promote fairness and mitigate bias.

**Steps:**
1. Build a graph from the Twitter dataset: nodes represent users, edges represent connections.
2. Detect communities and border nodes between communities.
3. Use CrossWalk to adjust edge weights, increasing weights of edges connecting border nodes to encourage fair traversal.
4. Run Random Walk with Restart on the reweighted graph to obtain visitation scores.
5. Analyze results to identify influential nodes and their communities.

**Required libraries:**
- NetworkX for graph operations
- NumPy for numerical computations

Implement the following functions:


In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import community as community_louvain

**Description:** Build a NetworkX graph from an edge list file.

- Input: `file_path` (str) – path to the edge list file.
- Output: a `networkx.Graph` object.

In [2]:
def create_graph_from_edgelist(file_path):
    """
    Build a NetworkX weighted graph from an edge list file.

    Parameters:
    - file_path (str): Path to the edge list file. Each line should be in the form "node1 node2 weight".

    Returns:
    - G (networkx.Graph): The graph constructed from the edge list with edge weights.
    """
    G = nx.Graph()

    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 3:
                node1, node2, weight = parts
                G.add_edge(node1, node2, weight=float(weight))

    return G

**Description:** Detect communities in the graph using an appropriate clustering algorithm.

- Input: `G` (`networkx.Graph`).
- Output: list of sets, each representing a community.

In [3]:
def detect_communities(G):
    """
    Detect communities in the graph using the Louvain method.

    Parameters:
    - G (networkx.Graph): The input graph.

    Returns:
    - communities (list of sets): A list where each set contains the nodes in one community.
    """
    # Partition is a dict: node -> community_id
    partition = community_louvain.best_partition(G)
    
    # Organize nodes by community
    communities = {}
    for node, comm_id in partition.items():
        communities.setdefault(comm_id, set()).add(node)

    return list(communities.values())

**Description:** Identify border nodes that lie between different communities.

- Input: `G` (`networkx.Graph`).
- Input: `communities` (list of sets).
- Output: list of border node IDs.

In [4]:
def identify_border_nodes(G, communities):
    """
    Identify border nodes that lie between different communities.

    Parameters:
    - G (networkx.Graph): The input graph.
    - communities (list of sets): List of communities (each as a set of node IDs).

    Returns:
    - border_nodes (list): List of node IDs that are border nodes.
    """
    # Step 1: Create a node -> community_id map
    node_to_comm = {}
    for i, community in enumerate(communities):
        for node in community:
            node_to_comm[node] = i

    border_nodes = []

    # Step 2: Identify border nodes
    for node in G.nodes():
        node_comm = node_to_comm[node]
        for neighbor in G.neighbors(node):
            if node_to_comm[neighbor] != node_comm:
                border_nodes.append(node)
                break  # No need to check more neighbors
          
    return border_nodes

**Description:** Adjust edge weights to emphasize fairness across communities.

- Input: `G` (`networkx.Graph`).
- Input: `border_nodes` (list of nodes).
- Output: a data structure (e.g., dict or matrix) representing CrossWalk weights.

In [6]:
def crosswalk(G, border_nodes, boost_factor=2.0):
    """
    Adjust edge weights to emphasize fairness across communities.

    Parameters:
    - G (networkx.Graph): The input graph.
    - border_nodes (list): List of node IDs considered as border nodes.
    - boost_factor (float): The factor by which to increase weights of border-connected edges.

    Returns:
    - crosswalk_weights (dict): A dictionary {(u, v): new_weight} with adjusted weights.
    """
    border_set = set(border_nodes)
    crosswalk_weights = {}

    for u, v, data in G.edges(data=True):
        original_weight = data.get('weight', 1.0)
        if u in border_set or v in border_set:
            adjusted_weight = original_weight * boost_factor
        else:
            adjusted_weight = original_weight

        # Store edge as sorted tuple to match undirected nature
        edge_key = tuple(sorted((u, v)))
        crosswalk_weights[edge_key] = adjusted_weight

    return crosswalk_weights

**Description:** Execute Random Walk with Restart on the weighted graph.

- Input: `G` (`networkx.Graph`).
- Input: `alpha` (float) – restart probability.
- Input: `tol` (float) – convergence tolerance.
- Input: `max_iter` (int) – maximum iterations.
- Output: dict mapping node IDs to RWR scores.

In [7]:
def random_walk_with_restart(G, alpha=0.85, tol=1e-6, max_iter=100):
    """
    Execute Random Walk with Restart on a weighted graph.

    Parameters:
    - G (networkx.Graph): Weighted graph.
    - alpha (float): Restart probability (typically 0.85).
    - tol (float): Convergence tolerance.
    - max_iter (int): Maximum number of iterations.

    Returns:
    - scores_dict (dict): Mapping from node ID to RWR score (aggregated over all starts).
    """
    nodes = list(G.nodes())
    n = len(nodes)
    node_idx = {node: i for i, node in enumerate(nodes)}
    idx_node = {i: node for i, node in enumerate(nodes)}

    # Build weighted adjacency matrix
    A = np.zeros((n, n))
    for u, v, data in G.edges(data=True):
        i, j = node_idx[u], node_idx[v]
        weight = data.get('weight', 1.0)
        A[i, j] = A[j, i] = weight  # undirected

    # Normalize to get transition matrix P
    D = A.sum(axis=1)
    P = A / D[:, None]  # row-normalized

    # Aggregate RWR scores starting from each node
    total_scores = np.zeros(n)

    for start in range(n):
        r = np.zeros(n)
        r[start] = 1.0
        prev_r = np.zeros(n)

        for _ in range(max_iter):
            r_new = (1 - alpha) * P @ r + alpha * np.eye(n)[start]
            if np.linalg.norm(r_new - r, 1) < tol:
                break
            r = r_new

        total_scores += r

    # Normalize scores across all nodes
    total_scores /= n

    # Map back to node labels
    scores_dict = {idx_node[i]: total_scores[i] for i in range(n)}
    return scores_dict

**Description:** Summarize and report the analysis results.

- Input: `G` (`networkx.Graph`).
- Input: `communities` (list of sets).
- Input: `border_nodes` (list of nodes).
- Input: `rwr_scores` (dict of node scores).
- Output: visualizations or saved report.

In [8]:
def analyze_results(G, communities, border_nodes, rwr_scores, top_k=10, save_path="analysis.png"):
    """
    Summarize and report the analysis results visually and in text.

    Parameters:
    - G (networkx.Graph): The graph.
    - communities (list of sets): List of node sets for each community.
    - border_nodes (list): List of border node IDs.
    - rwr_scores (dict): Mapping from node ID to RWR score.
    - top_k (int): Number of top influential nodes to report.
    - save_path (str): File path to save the visualization.
    """

    # --- 1. Print top-k influential nodes ---
    print(f"\nTop {top_k} influential nodes by RWR score:")
    top_nodes = sorted(rwr_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
    for node, score in top_nodes:
        print(f"Node {node}: Score = {score:.5f}")

    # --- 2. Map node to community color ---
    node_to_community = {}
    for i, comm in enumerate(communities):
        for node in comm:
            node_to_community[node] = i

    colors = [node_to_community[n] for n in G.nodes()]
    sizes = [rwr_scores[n] * 5000 for n in G.nodes()]  # Scale scores for visibility

    # --- 3. Draw the graph ---
    pos = nx.spring_layout(G, seed=42)  # fixed layout for reproducibility
    plt.figure(figsize=(12, 10))
    
    # Regular nodes
    nx.draw_networkx_nodes(G, pos, 
                           node_size=sizes, 
                           node_color=colors, 
                           cmap=plt.cm.tab20, 
                           alpha=0.8)
    
    # Border nodes in red outline
    nx.draw_networkx_nodes(G, pos, 
                           nodelist=border_nodes, 
                           node_size=100, 
                           edgecolors='red', 
                           facecolors='none', 
                           linewidths=2)

    nx.draw_networkx_edges(G, pos, alpha=0.3)
    plt.title("Twitter Graph - RWR Influence & Communities")
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.show()

    print(f"\nGraph visualization saved to: {save_path}")

### Output Pipeline

#### Step 1: Create the graph

In [10]:
G = create_graph_from_edgelist("higgs-retweet_network.edgelist")

In [21]:
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

Number of nodes: 256491
Number of edges: 327374


#### Step 2: Detect communities


In [12]:
communities = detect_communities(G)
communities

[{'99029',
  '59314',
  '144212',
  '83780',
  '391916',
  '365524',
  '350421',
  '36071',
  '253808',
  '282823',
  '367526',
  '242717',
  '90046',
  '70064',
  '211719',
  '291396',
  '206337',
  '311113',
  '313161',
  '63719',
  '254701',
  '255622',
  '167291',
  '50347',
  '58778',
  '406554',
  '435003',
  '411804',
  '287366',
  '296492',
  '40250',
  '341677',
  '365319',
  '319841',
  '14585',
  '352746',
  '401138',
  '358020',
  '82679',
  '276575',
  '355758',
  '423007',
  '406304',
  '58017',
  '119824',
  '165775',
  '387744',
  '293899',
  '357063',
  '65150',
  '205491',
  '133253',
  '99117',
  '246726',
  '262617',
  '124738',
  '49637',
  '413842',
  '16516',
  '63989',
  '333345',
  '384058',
  '233807',
  '398456',
  '65615',
  '35304',
  '410214',
  '349638',
  '122298',
  '410639',
  '102122',
  '35146',
  '455164',
  '407096',
  '453041',
  '53355',
  '35037',
  '428573',
  '166660',
  '421929',
  '373447',
  '335099',
  '28088',
  '341990',
  '215084',
  '3

#### Step 3: Identify border nodes


In [ ]:
border_nodes = identify_border_nodes(G, communities)

35270

#### Step 4: Apply CrossWalk to reweight edges


In [17]:
new_weights = crosswalk(G, border_nodes)

#### Step 5: Apply new weights to the graph


In [18]:
for (u, v), weight in new_weights.items():
    G[u][v]['weight'] = weight

#### Step 6: Run Random Walk with Restart


In [19]:
rwr_scores = random_walk_with_restart(G)

MemoryError: Unable to allocate 490. GiB for an array with shape (256491, 256491) and data type float64

#### Step 7: Analyze and visualize the results


In [ ]:
analyze_results(G, communities, border_nodes, rwr_scores)